### Group Members:

- Giovanni Sergio Armido Parri, 16-919-144

# Assignment 3: Universal Function Approximator


The goal of this exercise is to train a two-layer fully-connected network to perform one-dimensional non-linear regression via gradient descent. To show the flexibility of the approach, three different functions will be approximated. First, the network and its gradient need to be implemented. Second, target data for three different functions will be generated. Finally, the training procedure will be applied to the data, and the resulting approximated function will be plotted together with the data samples.

## Network Implementation

A two-layer network is defined by parameters $\Theta=(\mathbf W^{(1)},\vec w^{(2)})$ that are split into $\mathbf W^{(1)}\in\mathbb R^{K\times {(D+1)}}$ for the first layer and $\vec w^{(2)}\in\mathbb R^{K+1}$ for the second layer. In our case, since we have only a single input, we have $D=1$.
For a given input $\vec x = (1, x)^T$, the network is implemented in three steps:

1. Compute the first layer output, aka, the activation: $\vec a_- = \mathbf W^{(1)} \vec x$
2. Apply the activation function for each element of $\vec a_- : \vec h_- = g(\vec a_-)$ and prepend the bias neuron $h_0=1$ to arrive at $\vec h$.
3. Compute the output of the network: $y = \vec w^{(2)}\ ^T\vec h$.

### Task 1  
Implement a function that returns the network output for a given input $\vec x$ and parameters $\Theta=(\mathbf W^{(1)}, \vec w^{(2)})$. Remember that the input of the function $\vec x = (1, x)^T$. Also remember to prepend $h_0=1$ in your implementation.

We use sigmoid $(\sigma)$ as the activation function:

\begin{equation*}
    \sigma(a) =  \frac{\mathrm{1} }{\mathrm{1} + e^{-a} }
\end{equation*}

Note:

1. Use the `numpy` to implement the sigmoid function.
2. Use `numpy.concatenate` or `numpy.insert` to prepend $h_0$.
3. Make use of `numpy.dot` to compute matrix-vector and vector-vector products.

In [1]:
import numpy as np
import math

def logistic(x):
  return 1./(1.+np.exp(-x))

def network(x, Theta):
  W1, w2 = Theta
  a_ = np.dot(W1,x)
  h_ = logistic(a_)
  h = np.insert(h_, 0,1.)
  y = np.dot(w2, h)
  return y, h

Test 1: Sanity Check
----------------------------

We select a specific number of hidden neurons and create the weights accordingly, using all zeros in the first layer and all ones in the second. The test case below assures that the function from Task 1 actually returns $11$ for those weights.

Note: your function should pass the test below.

In [2]:
K = 20
D = 1
W1 = np.zeros((K, D+1))
w2 = np.ones(K+1)
print(w2)
x = np.random.rand(D+1)
y,h = network(x, (W1, w2))
assert abs(11 - y) < 1e-6
print("Test passed")

[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
Test passed


## Gradient Implementation

In order to perform gradient descent, we need to define a loss function. As provided in the lecture, the $\mathcal J^{L_2}$ loss function is defined over a dataset $X=\{(\vec x^{[n]}, t^{[n]})\}$, that is defined as a list of tuples, as follows:

$$
   \mathcal J^{L_2} = \frac{1}{N}\sum_{i=1}^N (y^{[n]}-t^{[n]})^2
$$

where $y^{[n]}$ is the output of the network from Task 1 when inputting $\vec x^{[n]}$. Interestingly, however, we never explicitly need to compute the output of the loss function. It is only used to analytically compute the gradient as shown in the lecture.

The gradient is composed of two items, one for each layer. Particularly, for a given dataset $X$, the gradient of loss $J^{L_2}$ is defined as:

\begin{align}
  \frac{\partial \mathcal J}{\partial w_{kd}^{(1)}} &= \frac{2}{N} \sum\limits_{n=1}^N (y^{[n]}-t^{[n]}) w_{k}^{(2)} (1-h_{k}^{[n]}) h_{k}^{[n]} x_{d}^{[n]}\\
  \frac{\partial \mathcal J}{\partial w_{k}^{(2)}} &= \frac{2}{N} \sum\limits_{n=1}^N (y^{[n]}-t^{[n]}) h_{k}^{[n]}
\end{align}

### Task 2
Implement a function that returns the gradient as defined in $(1)$ and $(2)$ for a given dataset $X$, and given weights $\Theta=(\mathbf W^{(1)},\vec w^{(2)})$. Make sure that both parts of the gradient are computed. 

Hint:

1. Make use of the the function implemented in Task 1 where appropriate

Note:

  1. This is a slow implementation. We will see how to speed this up in the next lecture.
  2. You can make use of `numpy.zeros` to initialize the gradient.
  3. The outer product can be computed via `numpy.outer`.
  4. Remember that we used the $logistic$ activation function in our network.

In [3]:
def gradient(X, Theta):
  # split parameters for easier handling
  W1, w2 = Theta

  # define gradient with respect to both parameters
  dW1 = np.zeros_like(W1)
  dw2 = np.zeros_like(w2)
  print(dW1.shape)
  print(dw2.shape)
  # iterate over dataset
  for x, t in X:
    # compute the gradient
    y, h = network(x, Theta)

    dw2 += 2./len(X)*np.dot((y-t), h)



    for k in range(dW1.shape[0]):
      dW1[:,k]= 2./len(X)*np.outer((y-t)[k]*w2[k]*(1 - h[k])*h[k], x)

  # anything else?
  ...

  return dW1, dw2

## Gradient Descent

The procedure of gradient descent is the repeated application of two steps.
 
1. The gradient of loss $\nabla_{\Theta}\mathcal J^{L_2}$ is computed based on the current value of the parameters $\Theta=(\mathbf W^{(1)},\vec w^{(2)})$.
2. The weights are updated by moving a small step $\eta$ into the direction of the negative gradient:

$$
    \Theta = \Theta - \eta \nabla_{\Theta}\mathcal J
$$

As stopping criterion, we select the number of training epochs to be 10000.

### Task 3
Implement a function that performs gradient descent for a given dataset $X$, given initial parameters $\Theta$ and a given learning rate $\eta$ and returns the optimized parameters $\Theta^*$.

In [4]:
def gradient_descent(X, Theta, eta):
  epochs = 10000
  Theta_opt = np.copy(Theta)
  OPT = [Theta_opt]
  # perform iterative gradient descent
  for epoch in range(epochs):
    # compute the gradient
    grad = gradient(X, OPT[epoch])

    # update the parameters
    OPT.append(OPT[epoch]-eta*grad)

  # return optimized parameters
  return OPT[-1]


## Generate Datasets

In total, we will test our gradient descent function with three different datasets. Particularly, we approximate:

1. $X_1: t = \sin(3x)$ for $x\in[-1,1]$
2. $X_2: t = e^{-4x^2}$ for $x\in[-1,1]$
3. $X_3: t = x^5 + 3x^4 - 6x^3 -12x^2 + 5x + 129$ for $x\in[-4,2.5]$

### Task 4

Generate dataset $X_1$, for $N=60$ samples randomly drawn from range $x\in[-1,1]$. 
Generate data $X_2$ for $N=50$ samples randomly drawn from range $x\in[-1,1]$. 
Generate dataset $X_3$ for $N=200$ samples randomly drawn from range $x\in[-4,2.5]$. 
Implement all three datasets as lists of tuples: $\{(\vec x^{[n]}, t^{[n]})\mid 1\leq n\leq N\}$.

Note:

  1. You can use `numpy.random.uniform` to create uniformly distributed samples for $x$.
  2. Make sure that $\vec x = (1, x)^T$ for each sample.
  3. You can make use of `numpy.sin`, `numpy.exp` and `numpy.pow` to compute target values.

In [5]:


#set random numbers to be the same
np.random.seed(42)

# X1
x_1 = np.random.uniform(-1, 1, 60)
t_1 = np.sin(3*x_1)
x_1_bias = np.vstack((np.ones(60), x_1)).T

X1 = [(x_1_bias[i], t_1[i]) for i in range(60)]


# X2
x_2 =np.random.uniform(-1, 1, 50)
t_2 = np.exp(-4.*x_2)
x_2_bias  = np.vstack((np.ones(50), x_2)).T

X2 = [(x_2_bias[i], t_2[i]) for i in range(50)]

# X3
x_3 = np.random.uniform(-4, 2.5, 200)
t_3 = np.power(x_3, 5) + 3.*np.power(x_3, 4) -6.*np.power(x_3, 3)-12.*np.power(x_3, 2)+5.*x_3+129
x_3_bias = np.vstack((np.ones(200), x_3)).T

X3 = [(x_3_bias[i], t_3[i]) for i in range(200)]

### Test 2: Sanity Check

The test case below assures that the elements of each generated dataset are tuples with two elements, that the first element ($\vec x$) is a vector with two numbers, and that the second element ($t$) is a float.

In [6]:
assert all(
    isinstance(x, (tuple,list)) and
    len(x) == 2 and
    isinstance(x[0], (tuple,list,np.ndarray)) and
    len(x[0]) == 2 and
    isinstance(x[1], float)
    for X in (X1, X2, X3)
    for x in X
)

print('Test passed!')

Test passed!


## Function Approximation

Finally, we want to make use of our gradient descent implementation to approximate our functions. In order to see our success, we want to plot the functions together with the data.

### Task 5 (theoretical question)

When looking at the example plots in the exercise slides (exemplary solutions for the plotting Task 8), how many hidden neurons $K$ do we need in order to approximate the functions? Is there any difference between the three target functions? **Please discuss it in the markdown cell below:**

...

In [7]:
# Define the number of neurons for each target function based on your discussion
K1 = 2
K2 = 3
K3 = 3

### Task 6

For each of the datasets, randomly initialize the parameters $\Theta_1,\Theta_2,\Theta_3\in[-1,1]$ according to the number of hidden neurons estimated in Task 5.

Note:

  1. You can use `numpy.random.uniform` to initialize the weights.
  2. Make sure that the weight matrices are instantiated in the correct dimensions.

In [8]:
Theta1 = (np.random.uniform(-1,1, (K1, D+1)), np.random.uniform(-1,1, K1+1))
Theta2 = (np.random.uniform(-1,1, (K2, D+1)), np.random.uniform(-1,1, K2+1))
Theta3 = (np.random.uniform(-1,1, (K3, D+1)), np.random.uniform(-1,1, K3+1))

### Task 7

Call gradient descent function from Task 3 using the datasets $X_1, X_2, X_3$, the according created parameters $\Theta_1,\Theta_2,\Theta_3$. Store the resulting optimized weights $\Theta_1^*, \Theta_2^*, \Theta_3^*$ and the loss values.

**Please discuss first the appropriate learning rate $\eta$  for each of the three functions in the markdown cell below.** Based on your learning rates, you may neeed to optimize the learning rate $\eta$ for these functions. Do you see any differences? What are the best learning rates that you can find?

WARNING: Depending on the implementation, this might run for several minutes!

...

In [9]:
# Call gradient descent function using the datasets and initial weights that you created above
# Choose appropriate learning rates for each function

eta1 = 1e-5
eta2 = 1e-5
eta3 = 1e-5


Theta1_opt = gradient_descent(X1, Theta1, eta1)
Theta2_opt = gradient_descent(X2, Theta2, eta2)
Theta3_opt = gradient_descent(X3, Theta3, eta3)

print(Theta1_opt)
print(Theta2_opt)
print(Theta3_opt)



(2, 2)
(3,)


/Users/giovanniparri/.local/lib/python3.8/site-packages/numpy/lib/function_base.py:959: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, order=order, subok=subok, copy=True)


IndexError: invalid index to scalar variable.

## Data and Function Plotting

### Task 8

Implement a plotting function that takes a given dataset $X$, given parameters $\Theta$ and a defined range $R=[min,max]$. Each data sample $(x^{[n]},t^{[n]})$ of the dataset is plotted as an $''x''$. In order to plot the function that is approximated by the network, generate sufficient equally-spaced input values $x\in R$, compute the network output $y$ for these inputs, and plot them with a line.

Note:

  1. The dataset $X$ is defined as above, a list of tuples $(\vec x, t)$.
  2. Each input in the dataset is defined as $\vec x = (1,x)^T$.
  3. Equidistant points can be obtained via `numpy.arange`.

In [ ]:
from matplotlib import pyplot
def plot(X, Theta, R):
  # first, plot data samples
  pyplot.plot(..., ..., "rx", label="Data")

  # define equidistant points from min (R[0]) to max (R[1]) to evaluate the network
  x = ...
  # compute the network outputs for these values
  y = ...
  # plot network approximation
  pyplot.plot(x,y,"k-", label="network")
  pyplot.legend()

### Task 9

For each of the datasets and their according optimized parameters, call the plotting function from Task 8. Use range $R=[-1.5,1.5]$ for dataset $X_1$ and $X_2$, and range $R=[-5,4]$ for dataset $X_3$. Note that the first element of range $R$ should be the lowest $x$-location, and the second element of $R$ the highest value for $x$. Did the networks approximate the functions? What can we do if not?

In [ ]:
figure = pyplot.figure(figsize=(10,3))

# plot first function
pyplot.subplot(131)
...

# plot second function
pyplot.subplot(132)
...

# plot third function
pyplot.subplot(133)
...